HS CODE 10단위 영어 전처리

---
- Marian 모델로 성질통합분류코드명(한글) 영어로 번역
- 영문품목명 + 번역성질통합분류코드명(영어) = 품목명 컬럼 생성


## setting

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hs = pd.read_excel("/content/drive/MyDrive/관세청_HS부호_240101.xlsx")
hs

,HS부호,한글품목명,영문품목명,성질통합분류코드,성질통합분류코드명
0,101211000,농가 사육용,For farm breeding,11020101.0,(말)
1,101219000,기타,Other,11020101.0,(말)
2,101291000,경주말,Horses for racing,11020101.0,(말)
3,101299000,기타,Other,11020101.0,(말)
4,101300000,당나귀,Asses,11020190.0,(기타 산 동물)
...,...,...,...,...,...
12417,9706102000,악기류,Musical instruments,12050201.0,(수집품 및 골동품)
12418,9706103000,기타,Other,12050201.0,(수집품 및 골동품)
12419,9706901000,도자기류,Ceramics,12050201.0,(수집품 및 골동품)
12420,9706902000,악기류,Musical instruments,12050201.0,(수집품 및 골동품)


## preprocessing

In [ ]:
hs['성질통합분류코드명'] = hs['성질통합분류코드명'].str.replace(r'[\(\)]', '', regex=True)
hs

,HS부호,한글품목명,영문품목명,성질통합분류코드,성질통합분류코드명
0,101211000,농가 사육용,For farm breeding,11020101.0,말
1,101219000,기타,Other,11020101.0,말
2,101291000,경주말,Horses for racing,11020101.0,말
3,101299000,기타,Other,11020101.0,말
4,101300000,당나귀,Asses,11020190.0,기타 산 동물
...,...,...,...,...,...
12417,9706102000,악기류,Musical instruments,12050201.0,수집품 및 골동품
12418,9706103000,기타,Other,12050201.0,수집품 및 골동품
12419,9706901000,도자기류,Ceramics,12050201.0,수집품 및 골동품
12420,9706902000,악기류,Musical instruments,12050201.0,수집품 및 골동품


## Marian 번역 모델 적용

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import sentencepiece as spm

model_name = 'Helsinki-NLP/opus-mt-ko-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/842k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def translate(text):
    if pd.isnull(text):
        return None
    inputs = tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True)
    translated = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

In [ ]:
hs['성질통합분류코드명_영문'] = hs['성질통합분류코드명'].apply(translate)
hs

,HS부호,한글품목명,영문품목명,성질통합분류코드,성질통합분류코드명,성질통합분류코드명_영문
0,101211000,농가 사육용,For farm breeding,11020101.0,말,HORSE
1,101219000,기타,Other,11020101.0,말,HORSE
2,101291000,경주말,Horses for racing,11020101.0,말,HORSE
3,101299000,기타,Other,11020101.0,말,HORSE
4,101300000,당나귀,Asses,11020190.0,기타 산 동물,Other Animals From the World
...,...,...,...,...,...,...
12417,9706102000,악기류,Musical instruments,12050201.0,수집품 및 골동품,Collecting and antiques
12418,9706103000,기타,Other,12050201.0,수집품 및 골동품,Collecting and antiques
12419,9706901000,도자기류,Ceramics,12050201.0,수집품 및 골동품,Collecting and antiques
12420,9706902000,악기류,Musical instruments,12050201.0,수집품 및 골동품,Collecting and antiques


In [ ]:
hs['품목명'] = hs['영문품목명'] + ' belonging to ' + hs['성질통합분류코드명_영문']
hs.to_json("HS10전처리.json", orient='records')

In [ ]:
# 이후 수작업으로 전처리를 추가함